In [4]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import Dataset
import pandas as pd
#!wget -c https://repo.continuum.io/archive/Anaconda3-5.1.0-Linux-x86_64.sh
#!pip install scikit-learn

import sklearn
'''from google.colab import files
src = list(files.upload().values())[0]
open('.py','wb').write(src)
import mylib'''
import io
from torch.nn.utils.rnn import pad_sequence
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
from torch.nn import TransformerEncoder, TransformerEncoderLayer
import math
import copy

In [17]:
data = pd.read_csv('train_data_QUICK_START.csv')
sequences = data['sequence']
experiment = data['experiment_type']
#reactivity = data.iloc[:,4:210]

In [9]:
print(data)

         sequence_id                                           sequence  \
0       0000d87cab97  GGGAACGACUCGAGUAGAGUCGAAAAAGAUCGCCACGCACUUACGA...   
1       0000d87cab97  GGGAACGACUCGAGUAGAGUCGAAAAAGAUCGCCACGCACUUACGA...   
2       0001ca9d21b0  GGGAACGACUCGAGUAGAGUCGAAAAGGUGGCCGGCAGAAUCGCGA...   
3       0001ca9d21b0  GGGAACGACUCGAGUAGAGUCGAAAAGGUGGCCGGCAGAAUCGCGA...   
4       00021f968267  GGGAACGACUCGAGUAGAGUCGAAAACAUUGUUAAUGCCUAUAUUA...   
...              ...                                                ...   
335611  fffee332db3a  GGGAACGACUCGAGUAGAGUCGAAAAGAUAUGGAAGUAGAACUACC...   
335612  ffff7786573c  GGGAACGACUCGAGUAGAGUCGAAAAGAAGACGUGACGAAAAGUCA...   
335613  ffff7786573c  GGGAACGACUCGAGUAGAGUCGAAAAGAAGACGUGACGAAAAGUCA...   
335614  ffffa291ee37  GGGAACGACUCGAGUAGAGUCGAAAAGAUAUGGAAUGCGAAGAACC...   
335615  ffffa291ee37  GGGAACGACUCGAGUAGAGUCGAAAAGAUAUGGAAUGCGAAGAACC...   

       experiment_type                                       dataset_name  \
0              2A3_MaP

In [17]:
#dms, a3 = data[data.experiment_type=='DMS_MaP'].reset_index(drop=True), data[data.experiment_type=='2A3_MaP'].reset_index(drop=True)

In [62]:
'''#update NaN to csv file
m = reactivity.mean(axis=1)
for i, col in enumerate(reactivity):
    # using i allows for duplicate co lumns
    # inplace *may* not always work here, so IMO the next line is preferred
    # df.iloc[:, i].fillna(m, inplace=True)
    reactivity.iloc[:, i] = reactivity.iloc[:, i].fillna(m)
reactivity.to_csv('ReactValues',sep=',', index=False, encoding='utf-8')'''

In [6]:
reactivity = pd.read_csv('ReactValues')

In [7]:
#X_train, X_valid, y_train, y_valid = train_test_split(sequences, reactivity, test_size=0.2, random_state=42)
'''
def encode_sequence(sequence):
    encoding = {'A': 1, 'C': 2, 'G': 3, 'U': 4, '2a3_MaP':5, 'DMS_Map': 6}
    encoded_sequence = []
    for nucleotide in sequence:
        # Check if the nucleotide is in the encoding dictionary
        if nucleotide in encoding:
            encoded_sequence.append(encoding[nucleotide])
        else:
            # Handle unexpected characters (e.g., print a warning)
            print(f"Warning: Unexpected character '{nucleotide}' in sequence.")
            # You can choose to replace the unexpected character with a default encoding or ignore it

    # Convert the list of encoded nucleotides to a PyTorch tensor
    return torch.FloatTensor(encoded_sequence)

from torch.nn.utils.rnn import pad_sequence

# Encode sequences
X_train_encoded = [encode_sequence(seq) for seq in sequences]
#X_valid_encoded = [encode_sequence(seq) for seq in X_valid]

# Pad sequences to make them equal length
X_train_padded = torch.tensor(pad_sequence(X_train_encoded, batch_first=True))
#X_valid_padded = torch.tensor(pad_sequence(X_valid_encoded, batch_first=True))'''

C:\Users\matth\AppData\Local\Temp\ipykernel_20456\1473072840.py:25: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train_padded = torch.tensor(pad_sequence(X_train_encoded, batch_first=True))


In [109]:
# Split each string into characters, creating new columns
char_columns = sequences.str.split('', expand=True)

# Drop the first empty column (resulting from splitting an empty string at the beginning)
char_columns = char_columns.drop(columns=0)

# Rename the columns if needed
#char_columns.columns = [f'char_{i}' for i in char_columns.columns]

# Concatenate the new columns with the original DataFrame
df = pd.concat([experiments, char_columns], axis=1)
df = df.fillna(-1)
print(df)

       experiment_type  1  2  3  4  5  6  7  8  9  ... 198 199 200 201 202  \
0              2A3_MaP  G  G  G  A  A  C  G  A  C  ...  -1  -1  -1  -1  -1   
1              DMS_MaP  G  G  G  A  A  C  G  A  C  ...  -1  -1  -1  -1  -1   
2              2A3_MaP  G  G  G  A  A  C  G  A  C  ...  -1  -1  -1  -1  -1   
3              DMS_MaP  G  G  G  A  A  C  G  A  C  ...  -1  -1  -1  -1  -1   
4              2A3_MaP  G  G  G  A  A  C  G  A  C  ...  -1  -1  -1  -1  -1   
...                ... .. .. .. .. .. .. .. .. ..  ...  ..  ..  ..  ..  ..   
335611         DMS_MaP  G  G  G  A  A  C  G  A  C  ...  -1  -1  -1  -1  -1   
335612         2A3_MaP  G  G  G  A  A  C  G  A  C  ...  -1  -1  -1  -1  -1   
335613         DMS_MaP  G  G  G  A  A  C  G  A  C  ...  -1  -1  -1  -1  -1   
335614         2A3_MaP  G  G  G  A  A  C  G  A  C  ...  -1  -1  -1  -1  -1   
335615         DMS_MaP  G  G  G  A  A  C  G  A  C  ...  -1  -1  -1  -1  -1   

       203 204 205 206 207  
0       -1  -1  -1  -1  -1  
1    

In [125]:
class CustomDataset(Dataset):
    def __init__(self, x_data, y_data):
        self.sequences = x_data
        self.reactivities = y_data

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, index):
        rna_sequence = torch.tensor(self.process_input(self.sequences.iloc[index, :]).values)
        reactivity_values = torch.tensor(self.reactivities.iloc[index, :].values)


        return rna_sequence, reactivity_values

    def process_input(self, rna_sequence):
        # Implement your logic to process the input RNA sequence (e.g., convert to numerical representation)
        # Return the processed input sequence
        X_train_encoded = rna_sequence.apply(self.encode_sequence)
        return X_train_encoded
    
    def encode_sequence(self, item):
        encoding = {'A': 1, 'C': 2, 'G': 3, 'U': 4, '2A3_MaP':5, 'DMS_MaP': 6}
        if item == '':
             return np.nan
        elif item == -1:
             return -1
        if item in encoding:
            return encoding[item]
        else:
                # Handle unexpected characters (e.g., print a warning)
                print(f"Warning: Unexpected character '{item}' in sequence.")
        return np.nan

In [126]:
dataset = CustomDataset(df, reactivity)

batch_size = 32
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [127]:
next(iter(dataloader))

[tensor([[ 6.,  3.,  3.,  ..., -1., -1., -1.],
         [ 5.,  3.,  3.,  ..., -1., -1., -1.],
         [ 5.,  3.,  3.,  ..., -1., -1., -1.],
         ...,
         [ 5.,  3.,  3.,  ..., -1., -1., -1.],
         [ 6.,  3.,  3.,  ..., -1., -1., -1.],
         [ 6.,  3.,  3.,  ..., -1., -1., -1.]], dtype=torch.float64),
 tensor([[0.4298, 0.4298, 0.4298,  ..., 0.4298, 0.4298, 0.4298],
         [0.3289, 0.3289, 0.3289,  ..., 0.3289, 0.3289, 0.3289],
         [0.3063, 0.3063, 0.3063,  ..., 0.3063, 0.3063, 0.3063],
         ...,
         [0.3622, 0.3622, 0.3622,  ..., 0.3622, 0.3622, 0.3622],
         [0.3153, 0.3153, 0.3153,  ..., 0.3153, 0.3153, 0.3153],
         [0.3456, 0.3456, 0.3456,  ..., 0.3456, 0.3456, 0.3456]],
        dtype=torch.float64)]

In [4]:
class TransformerModel(nn.Module):

    def __init__(self, ntoken: int, d_model: int, nhead: int, d_hid: int,
                 nlayers: int, dropout: float = 0.5):
        super().__init__()
        self.model_type = 'Transformer'
        self.pos_encoder = PositionalEncoding(d_model, dropout)
        encoder_layers = TransformerEncoderLayer(d_model, nhead, d_hid, dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.embedding = nn.Embedding(ntoken, d_model)
        self.d_model = d_model
        self.linear = nn.Linear(d_model, ntoken)

        self.init_weights()

    def init_weights(self) -> None:
        initrange = 0.1
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.linear.bias.data.zero_()
        self.linear.weight.data.uniform_(-initrange, initrange)

    def forward(self, src: torch.Tensor, src_mask: torch.Tensor = None) -> torch.Tensor:
        """
        Arguments:
            src: Tensor, shape ``[seq_len, batch_size]``
            src_mask: Tensor, shape ``[seq_len, seq_len]``

        Returns:
            output Tensor of shape ``[seq_len, batch_size, ntoken]``
        """
        src = self.embedding(src) * math.sqrt(self.d_model)
        src = self.pos_encoder(src)
        if src_mask is None:
            """Generate a square causal mask for the sequence. The masked positions are filled with float('-inf').
            Unmasked positions are filled with float(0.0).
            """
            src_mask = nn.Transformer.generate_square_subsequent_mask(len(src)).to(device)
        output = self.transformer_encoder(src, src_mask)
        output = self.linear(output)
        return output

In [3]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Arguments:
            x: Tensor, shape ``[seq_len, batch_size, embedding_dim]``
        """
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)

In [1]:
'''# MultiHeadAttention is the process of giving attention to different parts of a sequence simultaneously to find which parts of a 
# sequence affect each other.
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        assert d_model % num_heads == 0, "d_model must be divisible by num_heads"
        
        self.d_model = d_model
        self.num_heads = num_heads
        self.d_k = d_model // num_heads
        # the model handles the queries, keys, and values automatically
        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)
        self.W_o = nn.Linear(d_model, d_model)
        
    def scaled_dot_product_attention(self, Q, K, V, mask=None):
        attn_scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)
        if mask is not None:
            attn_scores = attn_scores.masked_fill(mask == 0, -1e9)
        attn_probs = torch.softmax(attn_scores, dim=-1)
        output = torch.matmul(attn_probs, V)
        return output
        
    def split_heads(self, x):
        batch_size, seq_length, d_model = x.size()
        return x.view(batch_size, seq_length, self.num_heads, self.d_k).transpose(1, 2)
        
    def combine_heads(self, x):
        batch_size, _, seq_length, d_k = x.size()
        return x.transpose(1, 2).contiguous().view(batch_size, seq_length, self.d_model)
        
    def forward(self, Q, K, V, mask=None):
        Q = self.split_heads(self.W_q(Q))
        K = self.split_heads(self.W_k(K))
        V = self.split_heads(self.W_v(V))
        
        attn_output = self.scaled_dot_product_attention(Q, K, V, mask)
        output = self.W_o(self.combine_heads(attn_output))
        return output'''

NameError: name 'nn' is not defined

In [9]:
'''#Creates a network for passing information forward through the model using two linear layers and a non-linear activation function
class PositionWiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff):
        super(PositionWiseFeedForward, self).__init__()
        #2 linear and one nonlinear layer
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)
        self.relu = nn.ReLU()
#overwrites the nn.Module forward function to pass the data through the model
    def forward(self, x):
        return self.fc2(self.relu(self.fc1(x)))'''

In [6]:
'''#allows the model to understand the position of objects relative to a sequence
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_seq_length):
        super(PositionalEncoding, self).__init__()
        #creates an empty tensor
        pe = torch.zeros(max_seq_length, d_model)
        #math stuff I don't need to fully understand
        position = torch.arange(0, max_seq_length, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model))
        #uses a sin and cosine function with different frequencies to "plot the points"
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        
        self.register_buffer('pe', pe.unsqueeze(0))
        
    def forward(self, x):
        return x + self.pe[:, :x.size(1)]'''

In [9]:
'''# Encoder layer takes and processes the inputs
# Consists of a MultiHeadAttention, PositionWiseFeedForward, two normalization layers, and
class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(EncoderLayer, self).__init__()
        #Creates the attention layer and feed forward network for this encoder layer
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        #Two normalization layers to prevent overfitting of hidden layers during processing
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        # turns some input sequences to 0 to help prevent overfitting. The var dropout is the probability that this will happen, usually set between 0.2-0.5
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, mask):
        #Don't understand why x is inputted three times
        attn_output = self.self_attn(x, x, x, mask)
        x = self.norm1(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm2(x + self.dropout(ff_output))
        return x'''

In [10]:
'''# The decoder passes the data from the encoder through two layers of multihead attention layers (the first one masked) 
# and then a position wise feed forward layer
class DecoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(DecoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.cross_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, enc_output, src_mask, tgt_mask):
        # method of passing the information forward through the normalization layers and cross and self attention layers
        # May want to read more on the different kinds of multi-head-attention layers
        attn_output = self.self_attn(x, x, x, tgt_mask)
        x = self.norm1(x + self.dropout(attn_output))
        attn_output = self.cross_attn(x, enc_output, enc_output, src_mask)
        x = self.norm2(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm3(x + self.dropout(ff_output))
        return x'''

In [27]:
'''class Transformer(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout):
        super(Transformer, self).__init__()
        # The purpose of nn.embedding is to take discrete tokens, like words, and change them into more vectors that can be operated on. However,
        # we already changed the sequence of DNA to orthogonal vectors, so does that mean I should just take this out? (learn more)
        self.encoder_embedding = nn.Embedding(src_vocab_size, d_model)
        self.decoder_embedding = nn.Embedding(tgt_vocab_size, d_model)
        #gives information about the position to the model
        self.positional_encoding = PositionalEncoding(d_model, max_seq_length)
        #Creates the same amount of encoder and decoder layers based on the number of layers user specifies
        self.encoder_layers = nn.ModuleList([EncoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])
        self.decoder_layers = nn.ModuleList([DecoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])

        self.fc = nn.Linear(d_model, tgt_vocab_size)
        self.dropout = nn.Dropout(dropout)
# The purpose of the mask is to hide some of the parts of the sequence from each other to prevent the 
# model from gaining information on future or irrelevant parts of the sequence
    def generate_mask(self, src, tgt):
        #The model creates the mask automatically, I don't need to worry about the specifics
        src_mask = (src != 0).unsqueeze(1).unsqueeze(2)
        tgt_mask = (tgt != 0).unsqueeze(1).unsqueeze(3)
        seq_length = tgt.size(1)
        nopeak_mask = (1 - torch.triu(torch.ones(1, seq_length, seq_length), diagonal=1)).bool()
        tgt_mask = tgt_mask & nopeak_mask
        return src_mask, tgt_mask

    def forward(self, src, tgt):
        
        src_mask, tgt_mask = self.generate_mask(src, tgt)
        src_embedded = self.dropout(self.positional_encoding(self.encoder_embedding(src)))
        tgt_embedded = self.dropout(self.positional_encoding(self.decoder_embedding(tgt)))

        enc_output = src_embedded
        for enc_layer in self.encoder_layers:
            enc_output = enc_layer(enc_output, src_mask)

        dec_output = tgt_embedded
        for dec_layer in self.decoder_layers:
            dec_output = dec_layer(dec_output, enc_output, src_mask, tgt_mask)

        output = self.fc(dec_output)
        return output'''